In [1]:
import time
print('ライブラリインポート中')

import pandas as pd
import numpy as np
import random
from tqdm import tqdm

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# 自作関数
from dataload.data_load import data_load
from func.my_func \
import \
    fit_MTS, predict_MD, determine_threshold, predict_MTSBag_, make_result_df

from concurrent.futures import ThreadPoolExecutor


ライブラリインポート中


In [20]:
def wrap_multi_thread_run(args):
    return multi_thread_run(*args)

def multi_thread_run(X_train, y_train, SIZE):
    # bootstrap sampling
    resampled_data_x, resampled_data_y = resample(X_train, y_train, n_samples = SIZE)
    random_s = random.sample(
        list(resampled_data_x.columns), 
        len(resampled_data_x.columns) 
        if len(resampled_data_x.columns) < 7 
        else 7
        )
    resampled_data_x = resampled_data_x[random_s]

    _scaler, _inv_C, _columns = fit_MTS(resampled_data_x, resampled_data_y)

    y_train_pred = predict_MD(resampled_data_x, _scaler, _inv_C, _columns)

    _threshold = determine_threshold(resampled_data_y, y_train_pred)

    return _scaler, _inv_C, _columns, _threshold


In [22]:
start = time.time()

n_experiment = 10
data_list = [
    'yeast', 
    'wine', 
    'abalone', 
    'car',
    # 'cancer', 
    # 'letter'
    ]
print('MTSBag開始')

for data in data_list:
    print(data)
    result_df = pd.DataFrame(
                        columns=['AUC', 'accuracy', 'recall', 'Specificity', 'precision', 'gmeans', 'RS'],
                        index=range(n_experiment))
    for m in tqdm(range(n_experiment)):
        
        X, y = data_load(data)

        # パラメータ
        n_estimators = 10
        max_samples = 0.5

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # 実行するところ

        # K:再標本化の回数 SIZE:再標本化されたもののサンプルサイズ
        K = n_estimators
        SIZE = int(len(X) * max_samples)

        # 予測に必要なパラメータ
        select_columns = [0] * K
        result_scaler = [0] * K
        result_inv_C = [0] * K
        threshold = [0] * K

        for i in range(K):
            # bootstrap sampling
            resampled_data_x, resampled_data_y = resample(X_train, y_train, n_samples = SIZE)
            random_s = random.sample(
                list(resampled_data_x.columns), 
                len(resampled_data_x.columns) 
                if len(resampled_data_x.columns) < 7 
                else 7
                )
            resampled_data_x = resampled_data_x[random_s]

            result_scaler[i], result_inv_C[i], select_columns[i] = fit_MTS(resampled_data_x, resampled_data_y)

            y_train_pred = predict_MD(resampled_data_x, result_scaler[i], result_inv_C[i], select_columns[i])

            threshold[i] = determine_threshold(resampled_data_y, y_train_pred)
            
        # y_proba, y_pred = predict_MTSBag_(X_test, result_scaler, result_inv_C, select_columns, threshold, K)

        # result_df = make_result_df(result_df, y_test, y_pred, y_proba, m)
end = time.time()
delta = end - start
print('処理時間:{}s'.format(round(delta,3))) 

MTSBag開始
yeast


 20%|██        | 2/10 [00:04<00:16,  2.01s/it]c:\Users\baseb\Documents\github\修論\SMOTEMTSBag\MTSBag\code\func\my_func.py:75: RuntimeWarning: divide by zero encountered in double_scalars
  sum_MD += 1 / row[i]
100%|██████████| 10/10 [00:20<00:00,  2.02s/it]


wine


100%|██████████| 10/10 [00:03<00:00,  2.73it/s]


abalone


100%|██████████| 10/10 [01:13<00:00,  7.32s/it]


car


100%|██████████| 10/10 [00:24<00:00,  2.43s/it]

処理時間:121.324s


In [23]:

start = time.time()

n_experiment = 10
data_list = [
    'yeast', 
    'wine', 
    'abalone', 
    'car',
    # 'cancer', 
    # 'letter'
    ]
print('MTSBag開始')

for data in data_list:
    print(data)
    result_df = pd.DataFrame(
                        columns=['AUC', 'accuracy', 'recall', 'Specificity', 'precision', 'gmeans', 'RS'],
                        index=range(n_experiment))
    for m in tqdm(range(n_experiment)):
        
        X, y = data_load(data)

        # パラメータ
        n_estimators = 10
        max_samples = 0.5

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # 実行するところ

        # n_estimators:再標本化の回数 SIZE:再標本化されたもののサンプルサイズ
        SIZE = int(len(X) * max_samples)

        job_args = list(zip([X_train] * n_estimators, [y_train] * n_estimators, [SIZE] * n_estimators))

        #========計算処理========
        with ThreadPoolExecutor(12) as e:
            ret = e.map(wrap_multi_thread_run, job_args)
        sms_multi = [r for r in ret]
        
            
        # y_proba, y_pred = predict_MTSBag_(X_test, result_scaler, result_inv_C, select_columns, threshold, n_estimators)

        # result_df = make_result_df(result_df, y_test, y_pred, y_proba, m)
end = time.time()
delta = end - start
print('処理時間:{}s'.format(round(delta,3))) 


MTSBag開始
yeast


100%|██████████| 10/10 [00:32<00:00,  3.20s/it]


wine


100%|██████████| 10/10 [00:04<00:00,  2.01it/s]


abalone


100%|██████████| 10/10 [01:42<00:00, 10.26s/it]


car


100%|██████████| 10/10 [00:37<00:00,  3.76s/it]

処理時間:177.246s


In [25]:
from multiprocessing import Pool, cpu_count


start = time.time()

n_experiment = 10
data_list = [
    'yeast', 
    'wine', 
    'abalone', 
    'car',
    # 'cancer', 
    # 'letter'
    ]
print('MTSBag開始')

for data in data_list:
    print(data)
    result_df = pd.DataFrame(
                        columns=['AUC', 'accuracy', 'recall', 'Specificity', 'precision', 'gmeans', 'RS'],
                        index=range(n_experiment))
    for m in tqdm(range(n_experiment)):
        
        X, y = data_load(data)

        # パラメータ
        n_estimators = 10
        max_samples = 0.5

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # 実行するところ

        # n_estimators:再標本化の回数 SIZE:再標本化されたもののサンプルサイズ
        SIZE = int(len(X) * max_samples)

        job_args = list(zip([X_train] * n_estimators, [y_train] * n_estimators, [SIZE] * n_estimators))

        #========計算処理========
        p = Pool(processes=cpu_count()-1)
        ret = p.map(wrap_multi_thread_run, job_args)
        sms_multi = [r for r in ret]
        
            
        # y_proba, y_pred = predict_MTSBag_(X_test, result_scaler, result_inv_C, select_columns, threshold, n_estimators)

        # result_df = make_result_df(result_df, y_test, y_pred, y_proba, m)
end = time.time()
delta = end - start
print('処理時間:{}s'.format(round(delta,3))) 


MTSBag開始
yeast


  0%|          | 0/10 [00:00<?, ?it/s]

In [2]:
def task(_n):
    s = 0
    k = 0
    for i in range(1,_n+1):
        s+=i
        k+=1
        time.sleep(0.01)
    return s, k
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor

ns = list(np.arange(1,11)) 

start = time.time()

#========計算処理========
with ThreadPoolExecutor(12) as e:
    ret = e.map(task, ns)
sms_multi = [r for r in ret]
#=======================
  
end = time.time()
delta = end - start
print('処理時間:{}s'.format(round(delta,3)))  
sms_multi

処理時間:0.157s


[(1, 1),
 (3, 2),
 (6, 3),
 (10, 4),
 (15, 5),
 (21, 6),
 (28, 7),
 (36, 8),
 (45, 9),
 (55, 10)]

In [5]:
import concurrent.futures
import math

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))

if __name__ == '__main__':
    main()

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.